# Projeto para buscar os dados de balanço das empresas listadas na CVM




## 1. Bibliotecas

In [2]:
import pandas as pd
import requests
import os
import zipfile

## 2. Configurando os acessos

In [3]:
diretorio_atual = os.getcwd()

os.chdir(f"{diretorio_atual}/dados_cvm")

### 2.1 Obtendo os dados

In [4]:
ano = range(2010, 2023) # Período de aquisição dos dados

url_base = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/' # Local dos dados

for ano in ano:
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

### 2.2 Concatenando as informações

In [6]:
lista_demonstracoes_2010_2022 = [] #criando a lista para armazenar os dados

diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):
    if arquivo == ".ipynb_checkpoints": #condição para passar caso haja algum arquivo como oculto na pasta de checkpoint
        pass
    else:
        arquivo_zip = zipfile.ZipFile(arquivo)        
        for planilha in arquivo_zip.namelist():
            demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep=";", encoding = 'ISO-8859-1',
                                      dtype = {"ORDEM_EXEC": "category"})
            lista_demonstracoes_2010_2022.append(demonstracao)


In [8]:
base_dados = pd.concat(lista_demonstracoes_2010_2022)

base_dados

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
0,00.000.000/0001-91,2010-12-31,1,BCO BRASIL S.A.,1023,DFP,4822.0,2011-02-17,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2010-12-31,2,BCO BRASIL S.A.,1023,DFP,4823.0,2011-02-17,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DFP,6945.0,2011-04-30,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.208/0001-00,2010-12-31,1,BRB BCO DE BRASILIA S.A.,14206,DFP,5436.0,2011-03-22,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.000.208/0001-00,2010-12-31,2,BRB BCO DE BRASILIA S.A.,14206,DFP,6775.0,2011-04-29,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58618,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.02,Dividendos,0.0,S,2022-01-01,NaN
58619,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,847006.0,S,2021-01-01,NaN
58620,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,506861.0,S,2022-01-01,NaN
58621,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.05,Outros,0.0,S,2021-01-01,NaN


## 3.0 Tratando da base de dados

In [11]:
#Separando a coluna 'GRUPO_DFP' que trás duas informaçõe concatenadas
base_dados[['con_ind', 'tipo_item']] = base_dados['GRUPO_DFP'].str.split("-", expand = True) #separando no '-' e armazenando em 2 variáveis
base_dados['con_ind'] = base_dados['con_ind'].str.strip() #apagando os espaços em brando antes e após o hífen
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip() #apagando os espaços em brando antes e após o hífen

#Os dados constam o 'ÚLTIMO' e o 'PENÚLTIMO' balanço, como estes não interessam para a análise, serão excluídos.
base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']

In [12]:
# Filtrar os tipos de demonstrações financeiras através da coluna criada na etapa anterior, pegando so valores únicos
lista_demonstracoes_financeiras = base_dados['tipo_dem'].unique()
lista_demonstracoes_financeiras

array([nan, 'Balanço Patrimonial Ativo', 'Balanço Patrimonial Passivo',
       'Demonstração do Fluxo de Caixa (Método Direto)',
       'Demonstração do Fluxo de Caixa (Método Indireto)',
       'Demonstração das Mutações do Patrimônio Líquido',
       'Demonstração de Resultado Abrangente',
       'Demonstração do Resultado', 'Demonstração de Valor Adicionado'],
      dtype=object)

In [14]:
# Filtrar por empresas
lista_empresas = base_dados['DENOM_CIA'].unique()

lista_empresas

array(['BCO BRASIL S.A.', 'BRB BCO DE BRASILIA S.A.',
       'CENTRAIS ELET BRAS S.A. - ELETROBRAS', ...,
       'SOLVI ESSENCIS AMBIENTAL S.A.', 'RIO+ SANEAMENTO BL 3 S.A.',
       'NOVA SOCIEDADE DE NAVEGAÇÃO S.A.'], dtype=object)

In [15]:
# Filtrando os dados de uma empresa em específico, por exemplo o DRE da WEG

weg_dre = base_dados[(base_dados['DENOM_CIA'] == 'WEG S.A.') &
          (base_dados['tipo_dem'] == 'Demonstração do Resultado') &
          (base_dados['DS_CONTA'] == 'Receita de Venda de Bens e/ou Serviços') &
          (base_dados['cond_ind'] == 'DF Consolidado')]

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,...,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF,con_ind,tipo_item
1025,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410,DFP,4906.0,2011-02-23,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41091,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Consolidado - Balanço Patrimonial Ativo,...,ÚLTIMO,2010-12-31,1,Ativo Total,7511164.0,S,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
41093,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Consolidado - Balanço Patrimonial Ativo,...,ÚLTIMO,2010-12-31,1.01,Ativo Circulante,4794009.0,S,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
41095,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Consolidado - Balanço Patrimonial Ativo,...,ÚLTIMO,2010-12-31,1.01.01,Caixa e Equivalentes de Caixa,53971.0,S,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
41097,84.429.695/0001-11,2010-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Consolidado - Balanço Patrimonial Ativo,...,ÚLTIMO,2010-12-31,1.01.02,Aplicações Financeiras,2499025.0,S,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54626,84.429.695/0001-11,2022-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2022-12-31,7.08.04,Remuneração de Capitais Próprios,4208084.0,S,2022-01-01,NaN,DF Individual,Demonstração de Valor Adicionado
54628,84.429.695/0001-11,2022-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2022-12-31,7.08.04.01,Juros sobre o Capital Próprio,749120.0,S,2022-01-01,NaN,DF Individual,Demonstração de Valor Adicionado
54630,84.429.695/0001-11,2022-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2022-12-31,7.08.04.02,Dividendos,1503231.0,S,2022-01-01,NaN,DF Individual,Demonstração de Valor Adicionado
54632,84.429.695/0001-11,2022-12-31,1,WEG S.A.,5410,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,ÚLTIMO,2022-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,1955733.0,S,2022-01-01,NaN,DF Individual,Demonstração de Valor Adicionado
